In [2]:
#!pip install ..
!pip install emoji
import emoji

!pip list
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

!pip install seaborn
!pip install scikit-learn
!pip install optuna

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Package                   Version
------------------------- --------------
alembic                   1.15.2
annotated-types           0.7.0
anyio                     4.9.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.4
bleach                    6.2.0
blis                      1.3.0
catalogue                 2.0.10
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
cloudpathlib              0.21.0
colorlog                  6.9.0
comm                      0.2.2
confection                0.1.5
contourpy                 1.3.2
cycler                    0.12.1


In [3]:
# =============================================================================
# 1. Chargement des données, prétraitement et vectorisation (TF-IDF)
# =============================================================================

df=pd.read_csv('scitweets_balanced.tsv', sep='\t')
display (df.head())

#code etienne matrice
import re
import numpy as np

stopwords_df = pd.read_csv("StopWordsFrench.csv", sep=',', index_col=0)
# Conversion en liste. Selon la structure, ici on récupère les index comme stopwords.
french_stopwords = stopwords_df.index.tolist()

vectorizer = TfidfVectorizer(
    min_df = 5,
    max_df = 0.95,
    stop_words=french_stopwords,
    lowercase=False,
    ngram_range=(1,2),
    max_features=50000
)
X_tfidf = vectorizer.fit_transform(df['text'])

tfidf_df = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vectorizer.get_feature_names_out()
)

def is_valid_token(token):
    # Garder uniquement les mots en alphabet latin (minuscules/majuscules)
    # On rejette si c’est purement numérique ou contient des caractères spéciaux non latins
    if token.isdigit():
        return False
    if re.match(r'^[a-zA-Z]+$', token):  # Mot composé uniquement de lettres latines
        return True
    return False

# Filtrer les colonnes du DataFrame TF-IDF
filtered_columns = [token for token in tfidf_df.columns if is_valid_token(token)]
tfidf_filtered_df = tfidf_df[filtered_columns]

# Identifie les colonnes numériques (constituées uniquement de chiffres)
numeric_columns = [token for token in tfidf_df.columns if token.isdigit()]

# Crée une colonne 'numeric' qui est la somme des colonnes numériques
tfidf_df['numeric'] = tfidf_df[numeric_columns].sum(axis=1)

# Supprime les colonnes numériques individuelles
tfidf_df.drop(columns=numeric_columns, inplace=True)

# Supprimer aussi les colonnes avec des caractères non latins
filtered_columns = [token for token in tfidf_df.columns if is_valid_token(token)]
tfidf_filtered_df = tfidf_df[filtered_columns + ['numeric']]

print("Taille de la matrice TF-IDF :", tfidf_df.shape)
display(tfidf_df)



# (70% entraînement, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['science_related'], test_size=0.3, random_state=42, stratify=df['science_related'])
#print(X_test)

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
0,0.0,3.166700e+17,Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0
1,1.0,3.190909e+17,McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0
2,2.0,3.220309e+17,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3.0,3.226948e+17,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4.0,3.285244e+17,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0


Taille de la matrice TF-IDF : (1530, 1233)


,2VgsIg8Trp,AND,About,According,According to,Adversity,Adversity causes,Ain,All,America,...,years,you,you can,you know,you re,your,your heart,your mental,yours,numeric
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.218843,0.0,0.0,0.0,0.632992


In [6]:
# =============================================================================
# 2. Évaluation de base des modèles de classification
# =============================================================================

# Import des classifieurs
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

# Fonction de tracé de la matrice de confusion
def plot_confusion_matrix(y_true, y_pred, labels, title):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap="YlGnBu")
    plt.xlabel("Prédiction")
    plt.ylabel("Vrai label")
    plt.title(f"Matrice de confusion - {title}")
    plt.show()

# Dictionnaire des modèles de base
models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": LinearSVC(max_iter=10000),
    "K-NN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "Neural Network - MLP": MLPClassifier(max_iter=500)
}

results = []

print("\n======== Évaluation de base des modèles ========")
for name, model in models.items():
    print(f"\n🔍 {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    results.append((name, acc))
    
    print(f"✅ Accuracy: {acc:.4f}")
    print("📄 Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    
    # Pour tracer la matrice de confusion, on récupère l'ensemble des labels présents
    plot_confusion_matrix(y_test, y_pred, labels=np.unique(y), title=name)

# Comparaison graphique des performances
results_df = pd.DataFrame(results, columns=["Modèle", "Accuracy"])
results_df = results_df.sort_values("Accuracy", ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x="Accuracy", y="Modèle", palette="mako")
plt.title("Comparaison des performances des modèles (Accuracy)")
plt.xlabel("Accuracy")
plt.ylabel("Modèle")
plt.xlim(0, 1)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()


======== Évaluation de base des modèles ========

🔍 Naive Bayes
✅ Accuracy: 0.8431
📄 Classification Report:
              precision    recall  f1-score   support

           0     0.8015    0.9130    0.8537       230
           1     0.8985    0.7729    0.8310       229

    accuracy                         0.8431       459
   macro avg     0.8500    0.8430    0.8423       459
weighted avg     0.8499    0.8431    0.8423       459



NameError: name 'y' is not defined

In [ ]:
on va optuner tout les models pour savoir le quel est le mieux a bfaire cette aprem

In [6]:
# =============================================================================
# 3. Optimisation des hyperparamètres avec Optuna pour chaque modèle
# =============================================================================

import optuna
from sklearn.neural_network import MLPClassifier


# --- Objectif pour Neural Network (MLP) ---
def objective_mlp(trial):
    hidden_layer_sizes = trial.suggest_categorical("hidden_layer_sizes", [(50,), (100,), (50, 50), (100, 50)])
    learning_rate_init = trial.suggest_float("learning_rate_init", 1e-4, 1e-1, log=True)
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=learning_rate_init, max_iter=500)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

study_mlp = optuna.create_study(direction="maximize")
study_mlp.optimize(objective_mlp, n_trials=400)
print("\n=== Résultats Optuna pour Neural Network (MLP) ===")
print("Meilleurs paramètres:", study_mlp.best_params)
print("Meilleure accuracy obtenue:", study_mlp.best_value)

[I 2025-04-16 11:00:15,592] A new study created in memory with name: no-name-734e7c23-c125-4290-bb2f-8673ddda75eb
[W 2025-04-16 11:00:15,595] Trial 0 failed with parameters: {'hidden_layer_sizes': (100, 50), 'learning_rate_init': 0.0004709221943468913} because of the following error: NameError("name 'X_train' is not defined").
Traceback (most recent call last):
  File "/Users/salhinina/Desktop/mon_env/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/pf/x8z57h4121s6cmtp02twbdd00000gn/T/ipykernel_63676/2207945189.py", line 14, in objective_mlp
    model.fit(X_train, y_train)
              ^^^^^^^
NameError: name 'X_train' is not defined
[W 2025-04-16 11:00:15,597] Trial 0 failed with value None.


NameError: name 'X_train' is not defined

In [ ]:
# --- Objectif pour Naive Bayes ---
def objective_nb(trial):
    alpha = trial.suggest_float("alpha", 1e-3, 1.0, log=True)
    model = MultinomialNB(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

study_nb = optuna.create_study(direction="maximize")
study_nb.optimize(objective_nb, n_trials=50)
print("\n=== Résultats Optuna pour Naive Bayes ===")
print("Meilleurs paramètres:", study_nb.best_params)
print("Meilleure accuracy obtenue:", study_nb.best_value)


# --- Objectif pour SVM (LinearSVC) ---
def objective_svm(trial):
    C = trial.suggest_float("C", 1e-3, 1e2, log=True)
    model = LinearSVC(C=C, max_iter=10000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

study_svm = optuna.create_study(direction="maximize")
study_svm.optimize(objective_svm, n_trials=50)
print("\n=== Résultats Optuna pour SVM ===")
print("Meilleurs paramètres:", study_svm.best_params)
print("Meilleure accuracy obtenue:", study_svm.best_value)



# --- Objectif pour XGBoost ---
def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "use_label_encoder": False,
        "eval_metric": "logloss"
    }
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_xgb, n_trials=50)
print("\n=== Résultats Optuna pour XGBoost ===")
print("Meilleurs paramètres:", study_xgb.best_params)
print("Meilleure accuracy obtenue:", study_xgb.best_value)